In [3]:
# INIT_TASK - Data Manipulation Challenge
# Author: Parva Mehta

import pandas as pd
import numpy as np
from datetime import datetime
import time

pd.set_option("display.max_columns", None)

In [4]:
# Part A

# Load data
cust = pd.read_csv("Customers.csv")
prod = pd.read_csv("Products.csv")
trans = pd.read_csv("Transactions.csv")

print(cust.shape, prod.shape, trans.shape)

(2000, 6) (150, 5) (10015, 7)


In [6]:
# Basic info
print(cust.dtypes)
print(prod.dtypes)
print(trans.dtypes)

cust.head(3), prod.head(3), trans.head(3)

customer_id        int64
customer_name     object
email             object
signup_date       object
country           object
age              float64
dtype: object
product_id        int64
product_name     object
category         object
supplier_id       int64
cost_price      float64
dtype: object
transaction_id      int64
customer_id         int64
product_id          int64
quantity          float64
price             float64
timestamp          object
payment_method     object
dtype: object


(   customer_id customer_name                  email signup_date    country  \
 0            1    Customer41  customer410@yahoo.com  2022-01-28         UK   
 1            2     Custome41   custome411@email.com  2023-10-28     Mexico   
 2            3    customer41  customer412@email.com  2023-04-05  Australia   
 
     age  
 0  30.0  
 1  29.0  
 2  69.0  ,
    product_id   product_name category  supplier_id  cost_price
 0           1  Product_OFF_0     Toys           28      113.16
 1           2  Product_SPO_1    Books           43      166.06
 2           3  Product_SPO_2    Books           47      128.13,
    transaction_id  customer_id  product_id  quantity   price  \
 0            4292          130          89       1.0  112.87   
 1            9107          271          98       1.0  211.90   
 2            7842           66          33       1.0  225.04   
 
              timestamp payment_method  
 0  2024-08-17 09:13:00    Credit Card  
 1  2023-11-17 20:50:00         PayP

In [7]:
# Missing & duplicates
print("Missing:\n", trans.isna().sum())
print("Duplicate transaction_id:", trans['transaction_id'].duplicated().sum())

Missing:
 transaction_id      0
customer_id         0
product_id          0
quantity           97
price             102
timestamp           0
payment_method    101
dtype: int64
Duplicate transaction_id: 15


In [8]:
# Timestamp handling
trans['timestamp'] = pd.to_datetime(trans['timestamp'], errors='coerce')
trans['hour'] = trans['timestamp'].dt.hour
trans['day'] = trans['timestamp'].dt.day_name()
trans['month'] = trans['timestamp'].dt.month_name()

print(trans['timestamp'].min(), "->", trans['timestamp'].max())

2023-01-01 00:39:00 -> 2024-12-31 22:18:00


In [9]:
# Part B
# Fill missing values (median/mode)
def fillna_smart(df):
    for c in df.columns:
        if df[c].dtype.kind in 'biufc':
            df[c] = df[c].fillna(df[c].median())
        else:
            if df[c].isna().any():
                df[c] = df[c].fillna(df[c].mode().iloc[0])
    return df

cust = fillna_smart(cust)
prod = fillna_smart(prod)
trans = fillna_smart(trans)

In [10]:
# Revenue & merge
trans['quantity'] = pd.to_numeric(trans['quantity'], errors='coerce').fillna(0).astype(int)
trans['price'] = pd.to_numeric(trans['price'], errors='coerce').fillna(0)
trans['revenue'] = trans['quantity'] * trans['price']

full = trans.merge(cust, on='customer_id', how='left')
full = full.merge(prod, on='product_id', how='left')
full['cost_price'] = pd.to_numeric(full['cost_price'], errors='coerce')
full['profit_margin'] = np.where(full['price']!=0,
                                 (full['price']-full['cost_price'])/full['price']*100,
                                 np.nan)

full.head(3)

,transaction_id,customer_id,product_id,quantity,price,timestamp,payment_method,hour,day,month,revenue,customer_name,email,signup_date,country,age,product_name,category,supplier_id,cost_price,profit_margin
0,4292,130,89,1,112.87,2024-08-17 09:13:00,Credit Card,9,Saturday,August,112.87,Customer_129,customer_129129@yahoo.com,2023-12-20,Canada,40.0,Product_HOM_88,Books,2,59.72,47.089572
1,9107,271,98,1,211.90,2023-11-17 20:50:00,PayPal,20,Friday,November,211.90,Customer_270,customer_270270@email.com,2022-11-09,India,21.0,Product_BOO_97,Home & Garden,26,84.89,59.938650
2,7842,66,33,1,225.04,2023-03-21 23:23:00,Bank Transfer,23,Tuesday,March,225.04,Customer_65,customer_6565@email.com,2023-08-08,Canada,62.0,Product_BOO_32,Electronics,22,103.61,53.959296


In [11]:
# Part C
# C1: customer metrics
cust_rev = full.groupby('customer_id').agg({
    'revenue':'sum',
    'transaction_id':'nunique',
    'revenue':'mean'
}).rename(columns={'revenue':'avg_transaction_value','transaction_id':'num_transactions'})

cat_top = full.groupby(['customer_id','category']).size().reset_index(name='cnt')
cat_top = cat_top.sort_values(['customer_id','cnt'],ascending=[True,False]).drop_duplicates('customer_id')
cust_rev = cust_rev.merge(cat_top[['customer_id','category']],on='customer_id',how='left')
cust_rev = cust_rev.sort_values('avg_transaction_value',ascending=False)
cust_rev.head()

,customer_id,avg_transaction_value,num_transactions,category
88,89,9670.622000,5,Automotive
1665,1970,4503.700000,1,Automotive
557,588,4213.780263,1,Food & Beverage
1436,1694,3825.408571,7,Automotive
1676,1982,3564.600000,1,Office


In [12]:
# C2: monthly stats
full['month_period'] = full['timestamp'].dt.to_period('M').dt.to_timestamp()
m = full.groupby('month_period').agg(
    total_revenue=('revenue','sum'),
    unique_customers=('customer_id','nunique'),
    num_orders=('transaction_id','nunique')
).reset_index()
m['avg_order_value'] = m['total_revenue']/m['num_orders']
m['mom_growth'] = m['total_revenue'].pct_change()*100
m.head()

,month_period,total_revenue,unique_customers,num_orders,avg_order_value,mom_growth
0,2023-01-01,193405.812589,349,485,398.774871,NaN
1,2023-02-01,165296.517903,313,427,387.111283,-14.533842
2,2023-03-01,183939.273323,341,460,399.867985,11.278372
3,2023-04-01,183610.501489,359,475,386.548424,-0.178739
4,2023-05-01,195409.945890,344,450,434.244324,6.426345


In [13]:
# C3: product performance
p = full.groupby(['product_id','product_name']).agg(
    revenue=('revenue','sum'),
    qty=('quantity','sum'),
    pm=('profit_margin','mean')
).reset_index()

print("Top by revenue:\n", p.sort_values('revenue',ascending=False).head(10))
print("\nTop by qty:\n", p.sort_values('qty',ascending=False).head(10))
print("\nTop by margin:\n", p.sort_values('pm',ascending=False).head(10))

Top by revenue:
      product_id     product_name       revenue   qty          pm
97          101  Product_OFF_100  95005.120000   431   42.570323
58           60   Product_OFF_59  78366.194739  1115  -28.056624
46           48   Product_OFF_47  71998.077896   542  -15.166955
112         116  Product_CLO_115  66416.407761  1209 -423.012763
89           92   Product_ELE_91  65408.500000   182   42.517918
121         125  Product_AUT_124  56546.750931   617  -10.584249
73           75   Product_ELE_74  55931.400000   160   41.706567
114         118  Product_AUT_117  55178.950000   215   46.104987
104         108  Product_OFF_107  54293.740000   162   43.971692
7             8    Product_TOY_7  52609.010000   250   43.239548

Top by qty:
      product_id     product_name       revenue   qty          pm
39           41   Product_SPO_40  12267.912999  1751   31.836810
36           38   Product_SPO_37  35250.288965  1475 -226.883944
37           39   Product_BOO_38  36993.146321  1252  -69.7

In [24]:
# C4: RFM
latest = full['timestamp'].max()
rfm = full.groupby('customer_id').agg(
    last=('timestamp','max'),
    freq=('transaction_id','nunique'),
    money=('revenue','sum')
).reset_index()
rfm['recency'] = (latest - rfm['last']).dt.days

def bin3(x, labels=('Low','Med','High')):
    if x.nunique()==1: return pd.Series([labels[0]]*len(x))
    return pd.qcut(x,3,labels=labels)

rfm['R'] = bin3(rfm['recency'],('High','Med','Low'))
rfm['F'] = bin3(rfm['freq'])
rfm['M'] = bin3(rfm['money'])
rfm['Segment'] = rfm['R'].astype(str) + ' ' + rfm['F'].astype(str) + ' ' + rfm['M'].astype(str)
rfm.head()

,customer_id,last,freq,money,recency,R,F,M,Segment
0,1,2024-02-15 05:18:00,5,1424.99,320,Low,Med,Med,Low Med Med
1,2,2024-11-23 13:14:00,3,641.99,38,High,Low,Low,High Low Low
2,3,2024-11-23 03:16:00,30,11180.42,38,High,High,High,High High High
3,4,2024-07-11 23:40:00,8,4383.63,172,Med,High,High,Med High High
4,5,2024-08-09 23:01:00,5,1920.98,143,Med,Med,Med,Med Med Med


In [26]:
# Part D

def find_suspicious_transactions(df):
    cond1 = (df['quantity']>100) & (df['price']<10)
    df['cust_hr'] = df['customer_id'].astype(str)+'|'+df['timestamp'].dt.strftime('%Y-%m-%d %H')
    cond2 = df.groupby('cust_hr')['transaction_id'].transform('count')>3
    a = df.loc[cond1, ['transaction_id','customer_name']].assign(reason='high_qty_low_price')
    b = df.loc[cond2, ['transaction_id','customer_name']].assign(reason='>3_in_hr')
    return pd.concat([a,b]).drop_duplicates('transaction_id').reset_index(drop=True)

sus = find_suspicious_transactions(full)
sus.head()

,transaction_id,customer_name,reason
0,5808,Customer_1893,high_qty_low_price
1,3269,Customer_622,high_qty_low_price
2,48,Customer_1457,high_qty_low_price
3,3156,Customer_1104,high_qty_low_price
4,8549,Customer_169,high_qty_low_price


In [27]:
# Rolling 7-day avg revenue
rev = full.set_index('timestamp').resample('D')['revenue'].sum()
rev = rev.to_frame('daily_rev')
rev['7d_avg'] = rev['daily_rev'].rolling(7,min_periods=1).mean()
rev.head(10)

,daily_rev,7d_avg
timestamp,,
2023-01-01,8450.390000,8450.390000
2023-01-02,5574.010000,7012.200000
2023-01-03,3094.810000,5706.403333
2023-01-04,3792.090000,5227.825000
2023-01-05,4810.720000,5144.404000
2023-01-06,4537.730000,5043.291667
2023-01-07,4908.880000,5024.090000
2023-01-08,6719.040000,4776.754286
2023-01-09,12454.351883,5759.660269


In [28]:
# Feature Engineering
feat = full.groupby('customer_id').agg(
    signup=('signup_date','first'),
    first_tx=('timestamp','min')
).reset_index()
feat['signup'] = pd.to_datetime(feat['signup'],errors='coerce')
feat['days_since_signup'] = (feat['first_tx']-feat['signup']).dt.days

temp = full.sort_values(['customer_id','timestamp'])
temp['prev'] = temp.groupby('customer_id')['timestamp'].shift()
temp['gap'] = (temp['timestamp']-temp['prev']).dt.days
freq = temp.groupby('customer_id')['gap'].mean().reset_index().rename(columns={'gap':'avg_days_btwn'})

catdiv = full.groupby('customer_id')['category'].nunique().reset_index().rename(columns={'category':'cat_div'})
paypref = full.groupby('customer_id')['payment_method'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan).reset_index().rename(columns={'payment_method':'pref_payment'})

feat = feat.merge(freq,on='customer_id',how='left').merge(catdiv,on='customer_id').merge(paypref,on='customer_id')
feat.head()

,customer_id,signup,first_tx,days_since_signup,avg_days_btwn,cat_div,pref_payment
0,1,2022-01-28,2023-02-05 16:03:00,373,93.250000,3,PayPal
1,2,2023-10-28,2023-10-30 06:36:00,2,194.500000,2,Bank Transfer
2,3,2023-04-05,2023-02-04 06:44:00,-60,22.137931,3,Bank Transfer
3,4,2022-01-07,2023-04-28 16:11:00,476,62.285714,3,Bank Transfer
4,5,2022-12-15,2023-05-19 02:58:00,155,111.750000,2,Bank Transfer


In [31]:
# Part E

# Old loop (slow)
# result = []
# for _,r in cust.iterrows():
#     if r['age']>25 and r['country']=='USA': result.append(r['customer_id'])

# Vectorized
vec = cust.loc[(cust['age']>25)&(cust['country']=='USA'),'customer_id']

In [34]:
# CLV function
def customer_lifetime_value(df, cid, discount=0.1):
    sub = df[df['customer_id']==cid].dropna(subset=['timestamp'])
    if sub.empty: return 0
    first = sub['timestamp'].min()
    months = (sub['timestamp'].dt.year-first.year)*12 + (sub['timestamp'].dt.month-first.month)
    factor = 1/(1+discount)
    return (sub['revenue']*(factor**months)).sum()

# Example - customer_lifetime_value(full, 1001)

In [36]:
# Saving all key outputs
cust_rev.to_csv("customer_metrics.csv",index=False)
m.to_csv("monthly.csv",index=False)
rfm.to_csv("rfm.csv",index=False)
sus.to_csv("suspicious.csv",index=False)
feat.to_csv("features.csv",index=False)